In [2]:
# # from https://realpython.com/python-web-scraping-practical-introduction/

# from requests import get
# from requests.exceptions import RequestException
# from contextlib import closing
# from bs4 import BeautifulSoup

In [3]:
from IPython.display import HTML
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
from law_scraper import *

In [5]:
target_url = 'https://www.admin.ch/opc/de/classified-compilation/19995395/index.html'
raw_html = simple_get(target_url)

In [6]:
html = BeautifulSoup(raw_html, 'html.parser')
lawcontent = html.find(id="lawcontent")

# toolbar = lawcontent.find(id='toolbar')
# toolbar.decompose()

In [7]:
deleted = 0
# delete_span_classes = ["context-menu", 'expanderComparator']
delete_config = [dict(id='toolbar'),
                 dict(name='span', class_="context-menu"), 
                 dict(name='span', class_='expanderComparator'),
                 dict(name='span'),
                 dict(name='div', id="comparison"),
                ]

for dc in delete_config:
    for tag in lawcontent.find_all(**dc):
        if tag.string is None or not tag.string.strip():      
            tag.decompose()
            deleted += 1
        else:
            print(f'Skipped deleting: {tag.string}')
print(f'Deleted: {deleted}')

Deleted: 592


In [8]:
delete_tags = []

sv_text = lawcontent.find(text='Sachverzeichnis')
if sv_text is not None:
    sv = sv_text.parent.parent
    sv_a = sv.find_previous_sibling('a')
    sv_div = sv.find_next_sibling('div')

    delete_tags += [sv, sv_div, sv_a]

    
sv_text = lawcontent.find(text='Inhaltsverzeichnis')
if sv_text is not None:
    sv = sv_text.parent.parent
    sv_a = sv.find_previous_sibling('a')
    sv_div = sv.find_next_sibling('div')

    delete_tags += [sv, sv_div, sv_a]

    
for tag in delete_tags:
    tag.decompose()

In [9]:
div_c = []
div_c += lawcontent.find_all('div', class_='collapseableArticle')
div_c += lawcontent.find_all('div', class_='collapseable')

# These are footnotes markings
div_c += lawcontent.find_all('div', class_='fns')

# These are all links including some which can identify articles
div_c += lawcontent.find_all('a')

print(len(div_c))
for d in div_c:
    d.unwrap()
    
    
div_c = []
div_c += lawcontent.find_all('div')

print(len(div_c))
for d in div_c:
    d.unwrap()

2272
3


In [222]:
# print(lawcontent.prettify())

In [11]:
# HTML(str(lawcontent))

In [12]:
lawcontent.name = 'body'
lawcontent.attrs = {}

In [17]:
fn = './bundesverfassung_raw.html'
# content = lawcontent.encode(formatter="html")
content = lawcontent.prettify(encoding='utf-8', formatter="html")

with open(fn, 'wb') as f:
    f.write(content)

In [16]:
!start ./bundesverfassung_raw.html

In [18]:
!pandoc bundesverfassung_raw.html -o bundesverfassung.rst
!pandoc bundesverfassung.rst -o bundesverfassung.html
# !start bundesverfassung.html

In [22]:
import shutil

# shutil.copyfile('bundesverfassung.html', '../bundesverfassung.html')
shutil.copyfile('bundesverfassung.rst', '../bundesverfassung.rst')

'../bundesverfassung.rst'